In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

custom_headers = {
    "Accept-language": "en-GB,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "User-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15",
}

def get_soup(url):
    response = requests.get(url, headers=custom_headers)

    if response.status_code != 200:
        print(f"Error in getting webpage: Status code {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, "lxml")
    return soup

def get_reviews(soup):
    review_elements = soup.select("div.review")

    scraped_reviews = []

    for review in review_elements:
        r_author_element = review.select_one("span.a-profile-name")
        r_author = r_author_element.text.strip() if r_author_element else None

        r_rating_element = review.select_one("i.review-rating")
        r_rating = r_rating_element.text.strip().replace("out of 5 stars", "") if r_rating_element else None

        r_title_element = review.select_one("a.review-title")
        r_title_span_element = r_title_element.select_one("span:not([class])") if r_title_element else None
        r_title = r_title_span_element.text.strip() if r_title_span_element else None

        r_content_element = review.select_one("span.review-text")
        r_content = r_content_element.text.strip() if r_content_element else None

        r_date_element = review.select_one("span.review-date")
        r_date = r_date_element.text.strip() if r_date_element else None

        r_verified_element = review.select_one("span.a-size-mini")
        r_verified = r_verified_element.text.strip() if r_verified_element else None

        r_image_element = review.select_one("img.review-image-tile")
        r_image = r_image_element.attrs["src"] if r_image_element else None

        r = {
            "author": r_author,
            "rating": r_rating,
            "title": r_title,
            "content": r_content,
            "date": r_date,
            "verified": r_verified,
            "image_url": r_image
        }

        scraped_reviews.append(r)

    return scraped_reviews

def main():
    search_url = "https://www.amazon.in/Intel-i9-13900K-Desktop-Processor-P-cores/product-reviews/B0BCF54SR1/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
    soup = get_soup(search_url)

    if soup:
        data = get_reviews(soup)
        df = pd.DataFrame(data=data)

        df.to_csv("amz.csv", index=False)  # Ensure index is not saved as a column in CSV

        print("CSV file 'amz.csv' successfully created.")
    else:
        print("Failed to retrieve data. Check the URL or network connection.")

if __name__ == '__main__':
    main()

CSV file 'amz.csv' successfully created.
